Indicadores de Secretaría del Trabajo y Fomento al Empleo

Tasa de informalidad laboral (REVISAR)
Tasa de participación económica de las mujeres*
Tasa de condiciones críticas de ocupación
Indicador de Productividad Laboral


In [1]:
if(! require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr, janitor, stringr, readxl, writexl, tools,dplyr, purrr)

Cargando paquete requerido: pacman

Warning message:
"package 'pacman' was built under R version 4.4.2"


DESCARGA Y LECTURA DE DATOS

In [2]:
#Crear carpeta para almacenar los microdatos descagados del INEGI
dir.create('Microdatos', showWarnings = FALSE)

#Crear carpetas para almacenar los resultados
dir.create('PGD', showWarnings = FALSE)

In [3]:
# Directorios principales
dir_datos <- 'Microdatos'

# Verificar si existen antes de intentar crearlos
if (!dir.exists(dir_datos)) {
    dir.create(dir_datos, recursive = TRUE)
}


# Verificar que los directorios realmente existen después de crearlos
print(file.exists(dir_datos))  # Debe devolver TRUE

[1] TRUE


In [4]:
descargar_enoe_csv <- function() {
    options(timeout = 90000)
    data_list <- list()

    descargar_extraer <- function(url, year_str, trim) {
        zip_path <- file.path('Microdatos', basename(url))
        download.file(url, zip_path, mode = 'wb')
        
        unzip_dir <- file.path('Microdatos', paste0('ENOE_', year_str, '_trim', trim))
        dir.create(unzip_dir, showWarnings = FALSE)
        unzip(zip_path, exdir = unzip_dir)

        unlink(zip_path) # Eliminar el archivo zip después de descomprimirlo
        csv_files <- list.files(unzip_dir, pattern = '\\.csv$', full.names = TRUE)
        
        if(length(csv_files) == 0) {
            warning('No se encontró el archivo CSV en ', unzip_dir)
            return(NULL) 
        }

        df_list <- list()

        for(csv_file in csv_files) {
            nombre <- file_path_sans_ext(basename(csv_file))
            tipo <- NA

            if (str_detect(nombre, 'COE1T?')) {
                tipo <- 'COE1'
                } else if (str_detect(nombre, 'COE2T?')) {
                    tipo <- 'COE2'
                    } else if (str_detect(nombre, 'HOGT?')) {
                        tipo <- 'HOG'
                        } else if (str_detect(nombre, 'SDEMT?')) {
                            tipo <- 'SDEM'
                            } else if (str_detect(nombre, 'VIVT?')) {
                                tipo <- 'VIV'
                                
                                }

            if(!is.na(tipo)) {
                df <- tryCatch({
                    read_csv(csv_file, show_col_types = FALSE) %>% clean_names()
                }, error = function(e) {
                    warning('Error al leer el archivo CSV de ', csv_file)
                    return(NULL) 
                })
                
                if(!is.null(df)) {
                    df_list[[tipo]] <- df
                }
            } 
                    
        }
        
    return(df_list)
    
    }
    
    #iterar los años y trimestres de descarga    
    for(year in 21:22) {
        year_str <- paste0('20', year)
        data_list[[year_str]] <- list()
        
        for(trim in 1:4) {
        url <- paste0('https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_n_20', year, '_trim', trim, '_csv.zip')
        
        df <- descargar_extraer(url, year_str, trim)

        if(!is.null(df)) {
            data_list[[year_str]][[paste0('trim', trim)]] <- df
        }
    }

    }
    
    for(year in 23:24) {
        year_str <- paste0('20', year)
        data_list[[year_str]] <- list()
        
        for(trim in 1:4) {
        url <- paste0('https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_20', year, '_trim', trim, '_csv.zip')
                
        df <- descargar_extraer(url, year_str, trim)
        
        if(!is.null(df)) {
            data_list[[year_str]][[paste0('trim', trim)]] <- df
        }
    
    }

    } 
    
    save(data_list, file = 'Microdatos/data_list.RData')


    return(data_list)

}

In [ ]:
invisible(descargar_enoe_csv())

Cargar la lista de datos guardada para no tener que descargarla nuevamente 

In [5]:
load("Microdatos/data_list.RData")

In [ ]:
#Confirmar que la estructura de las listas de datos estén bien
str(data_list$'2023'$'trim4'$'COE1')

Población informal en la Ciudad de México

In [7]:

# Definir los años y trimestres a trabajar
anios <- as.character(2021:2024)
trimestres <- paste0("trim", 1:4)

# Extraer y unir SDEM de todos los trimestres y años, estandarizando columnas a character
sdem_todos <- cross_df(list(anio = anios, trim = trimestres)) %>%
  mutate(data = map2(anio, trim, ~ {
    df <- data_list[[.x]][[.y]][["SDEM"]]
    
    if (!is.null(df)) {
      df <- df %>%
        mutate(across(everything(), as.character)) %>%  #todas las columnas como character
        mutate(anio = as.integer(.x), trimestre = .y)
    } else {
      NULL
    }
  })) %>%
  pull(data) %>%
  compact() %>%
  bind_rows()

Warning message:
"`cross_df()` was deprecated in purrr 1.0.0.
ℹ Please use `tidyr::expand_grid()` instead.
ℹ See <https://github.com/tidyverse/purrr/issues/768>."


In [8]:
sdem_todos <- sdem_todos %>%
  mutate(
    fac_tri = as.numeric(fac_tri),
    eda = as.integer(eda),
    ent = as.integer(ent)
  )

In [ ]:
Variables <- list() #Lista para filtrar y seleccionar las variables que necesitamos
PobTot <- list() #Lista para vaciar valores de pob_total
Informal <- list() #Lista para vaciar los valores de población ocupada en el sector informal

In [132]:
calculo_informal <- function(data_list) {
  for (year in names(data_list)) {
    for (trim in names(data_list[[year]])) {
      if (!'SDEM' %in% names(data_list[[year]][[trim]])) next

#calcular la población ocupada wn la cdmx  :) 
      df <- data_list[[year]][[trim]][['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(r_def == 0, (c_res == 1 | c_res == 3), edad >= 15 & edad <= 98) %>%
        filter(clase2 == 1, ent == 9) 
      
      # Guardar nombres y df completo por si se necesita
      Variables[[paste0("v", year, "_", trim)]] <<- names(df)
      PobTot[[paste0("pob_total_", year, "_", trim)]] <<- df



     # Calcular la población ocupada en el sector informal
   df_informal <- df %>%
        filter(emp_ppal == 1) %>%
        filter( (pos_ocu == 1 & seg_soc == 2) |
          (pos_ocu %in% c(2, 3, 4) & tue2 %in% c( 2, 5, 6))) %>%
        mutate(sector = "Informal")



      total_sector <- sum(df_informal$fac_tri, na.rm = TRUE)
      total_pob <- sum(df$fac_tri, na.rm = TRUE)
      porcentaje <- (total_sector / total_pob) * 100

      Informal[[paste0("informal_", year, "_", trim)]] <<- tibble(
        year = as.integer(year),
        trimestre = trim,
        sector = "Informal",
        total_sector = total_sector,
        pob_total = total_pob,
        porcentaje = porcentaje
      )
    }
  }
}

# Ejecutar función sin mostrar resultados intermedios
invisible(calculo_informal(data_list))


In [137]:
# Unir resultados finales
informal_final <- bind_rows(Informal) %>%
  mutate(trimestre = factor(trimestre, levels = c("trim1", "trim2", "trim3", "trim4"))) %>%
  arrange(year, trimestre)

# Agregar por año
informal_anual <- informal_final %>%
  group_by(year, sector) %>%
  summarise(
    total_sector = sum(total_sector, na.rm = TRUE),
    pob_total = sum(pob_total, na.rm = TRUE),
    porcentaje = (total_sector / pob_total) * 100,
    .groups = "drop"
  ) %>%
  arrange(year)


informal_anual <- informal_final %>%
  group_by(year, sector) %>%
  summarise(
    total_sector = sum(total_sector, na.rm = TRUE),
    pob_total = sum(pob_total, na.rm = TRUE),
    porcentaje = (total_sector / pob_total) * 100,
    .groups = "drop"
  ) %>%
  arrange(year)

# Mostrar resultado anual
print(informal_anual)



# A tibble: 4 × 5
   year sector   total_sector pob_total porcentaje
  <int> <chr>           <dbl>     <dbl>      <dbl>
1  2021 Informal      7957422  16846846       47.2
2  2022 Informal      8400414  18164783       46.2
3  2023 Informal      8560864  18772600       45.6
4  2024 Informal      8666509  19453987       44.5


In [ ]:
#CALCULO CORRECTO 

tpo4 <- list()

calculo_tpo4 <- function(data_list) {
  
  for (year in names(data_list)) {
    for (trim in names(data_list[[year]])) {
      
      if (!'SDEM' %in% names(data_list[[year]][[trim]])) next
      
      df_base <- data_list[[year]][[trim]][['SDEM']] %>%
        janitor::clean_names() %>%
        mutate(edad = as.numeric(eda)) %>%
        filter(
          r_def == 0,
          c_res %in% c(1, 3),
          edad >= 15 & edad <= 98,
          ent == 9,
          clase2 == 1  # Solo ocupados
        )

      # Total población ocupada
      total_ocupados <- df_base %>%
        summarise(total = sum(fac_tri, na.rm = TRUE)) %>%
        pull(total)

      # Aplicar filtro TPO4
      df_tpo4 <- df_base %>%
        filter(
          (tip_con == 5 & seg_soc == 2 & emp_ppal == 2) |
          (pos_ocu %in% c(1, 2, 3, 4) & emp_ppal == 1)
        )

      total_tpo4 <- sum(df_tpo4$fac_tri, na.rm = TRUE)

      porcentaje <- (total_tpo4 / total_ocupados) * 100

      tpo4[[paste0('promedio_', year, '_', trim)]] <<- tibble(
        year = as.integer(year),
        trimestre = trim,
        total_ocupados = total_ocupados,
        total_tpo4 = total_tpo4,
        porcentaje = porcentaje
      )
    }
  }
}

# Ejecutar función
invisible(calculo_tpo4(data_list))

# Combinar resultados
tpo4_final <- bind_rows(tpo4) %>%
  mutate(trimestre = factor(trimestre, levels = c('trim1', 'trim2', 'trim3', 'trim4'))) %>%
  arrange(year, trimestre)

print(tpo4_final)


# A tibble: 16 × 5
    year trimestre total_ocupados total_tpo4 porcentaje
   <int> <fct>              <dbl>      <dbl>      <dbl>
 1  2021 trim1            3870454    1794589       46.4
 2  2021 trim2            4117427    1939674       47.1
 3  2021 trim3            4393859    2168037       49.3
 4  2021 trim4            4465106    2144407       48.0
 5  2022 trim1            4385718    2023263       46.1
 6  2022 trim2            4534376    2180962       48.1
 7  2022 trim3            4546515    2168004       47.7
 8  2022 trim4            4698174    2152425       45.8
 9  2023 trim1            4693298    2169763       46.2
10  2023 trim2            4709095    2179459       46.3
11  2023 trim3            4712506    2191013       46.5
12  2023 trim4            4657701    2131762       45.8
13  2024 trim1            4715069    2192253       46.5
14  2024 trim2            4834239    2190661       45.3
15  2024 trim3            4894634    2215259       45.3
16  2024 trim4            501

Tasa de participación económica de las mujeres                                                                                                                                                                              Porcentaje que representa la población económicamente activa (PEA) respecto a la de 15 y más años de edad.


In [14]:
Variables <- list() #Lista para filtrar y seleccionar las variables que necesitamos
PobTot <- list() #Lista para vaciar valores de pob_total
PEAmujeres <- list() #Lista para vaciar los valores de población eco

In [15]:
# Lista para resultados
TPE_Mujeres <- list()

In [ ]:
calculo_tpe_mujeres <- function(data_list) {
  for (year in names(data_list)) {
    for (trim in names(data_list[[year]])) {
      if (!'SDEM' %in% names(data_list[[year]][[trim]])) next
      
      df <- data_list[[year]][[trim]][['SDEM']] %>%
        clean_names() %>%
        mutate(edad = as.numeric(eda),
               sexo = as.integer(sex)) %>%
        filter(r_def == 0, (c_res == 1 | c_res == 3), edad >= 15 & edad <= 98,
               sex == 2,  
               ent == 9)   

      # Población total mujeres 15+
      pob_total_muj <- sum(df$fac_tri, na.rm = TRUE)
      
      pea_muj <- df %>%
        filter(clase2 == 1) %>%
        summarise(total_pea = sum(fac_tri, na.rm = TRUE)) %>%
        pull(total_pea)
      
      tpe <- (pea_muj / pob_total_muj) * 100
      
      TPE_Mujeres[[paste0("tpe_", year, "_", trim)]] <<- tibble(
        year = as.integer(year),
        trimestre = trim,
        pob_total_muj = pob_total_muj,
        pea_muj = pea_muj,
        tpe_muj = tpe
      )
    }
  }
}

In [17]:
# Ejecutar función sin mostrar resultados intermedios
invisible(calculo_tpe_mujeres(data_list))

In [18]:
# Unir resultados finales
tpe_mujeres_final <- bind_rows(TPE_Mujeres) %>%
  mutate(trimestre = factor(trimestre, levels = c("trim1", "trim2", "trim3", "trim4"))) %>%
  arrange(year, trimestre)

In [19]:
# Mostrar resultados
print(tpe_mujeres_final)

# A tibble: 16 × 5
    year trimestre pob_total_muj pea_muj tpe_muj
   <int> <fct>             <dbl>   <dbl>   <dbl>
 1  2021 trim1           4168688 1691499    40.6
 2  2021 trim2           4151850 1772648    42.7
 3  2021 trim3           4174088 1970545    47.2
 4  2021 trim4           4152856 1982122    47.7
 5  2022 trim1           4148738 1946624    46.9
 6  2022 trim2           4163416 1992162    47.8
 7  2022 trim3           4214921 2009548    47.7
 8  2022 trim4           4205491 2116597    50.3
 9  2023 trim1           4171434 2066809    49.5
10  2023 trim2           4239135 2142635    50.5
11  2023 trim3           4202125 2108472    50.2
12  2023 trim4           4275915 2101821    49.2
13  2024 trim1           4339569 2168563    50.0
14  2024 trim2           4355221 2269963    52.1
15  2024 trim3           4349180 2293694    52.7
16  2024 trim4           4354909 2393929    55.0


Indicador de Productividad Laboral

In [ ]:
vab_cdmx <- read_csv("vab_cdmx.csv")

In [98]:
list.files()

[1] "escenarios_trabajo.ipynb"       "Indicadores_trabajo.ipynb"     
[3] "Microdatos"                     "PGD"                           
[5] "Sistema_Indicadores_ENOE.ipynb" "vab_cdmx.csv"

In [99]:
getwd()


[1] "c:/Users/ipdp_/Escritorio/Monserrat/TRABAJO PGD"

In [ ]:
vab_raw <- read_csv("vab_cdmx.csv")

In [108]:
vab_cdmx

Concepto,2003,2004,2005,2006,2007,2008,2009,2010,2011,⋯,2014,2015,2016,2017,2018,2019,2020,2021,2022 R,2023 P
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Millones de pesos a precios de 2018,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
a B.1bV - Valor agregado bruto,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
Estados Unidos Mexicanos,592478.460,607316.558,588558.452,623901.414,651139.000,656325.502,638074.598,653518.474,637703.495,⋯,718687.394,732003.814,753344.638,777962.381,797095.021,799476.894,808429.880,827208.787,840161.193,828041.916
Aguascalientes,6761.509,6688.170,7137.339,7459.947,7799.641,8428.549,8083.994,8504.018,9023.803,⋯,10207.219,10636.975,11049.605,11773.626,12196.068,12409.537,12714.026,13055.677,12947.315,12983.940
Baja California,15451.977,17897.244,18277.022,17538.554,16570.119,16201.753,18709.536,18920.280,18312.404,⋯,19265.785,20482.275,19277.960,21009.541,21408.907,21874.793,19920.240,20766.047,23073.243,23595.050
Baja California Sur,4126.691,3825.118,4459.798,4030.299,4275.192,3995.717,3862.262,4598.812,4557.858,⋯,5575.650,5874.223,6201.172,6049.484,6351.325,6347.865,6825.902,7604.291,7422.812,7341.275
Campeche,3334.985,4124.897,4238.917,4728.508,3522.368,4092.090,4255.562,4964.887,4552.409,⋯,5384.054,5767.381,6276.866,6263.550,6495.615,6235.737,6444.039,6832.877,5981.548,6121.068
Coahuila de Zaragoza,15048.462,15628.885,17451.523,16317.073,16879.549,16562.888,18017.221,18406.247,18793.706,⋯,18748.817,17863.108,16688.639,16818.627,17696.895,18285.792,19015.259,18894.364,19095.324,18623.403
Colima,7050.439,7328.194,6929.928,7530.338,7665.183,8065.372,6359.863,6663.656,7244.732,⋯,5915.154,5959.494,6616.223,6886.233,7344.258,7407.790,7629.655,7685.166,8379.885,8231.778


In [101]:
#Calcular población ocupada total en CDMX por año
ocupados_anual <- sdem_todos %>%
  filter(ent == 9, clase2 == 1, !is.na(fac_tri)) %>%
  group_by(year = anio) %>%
  summarise(pob_ocupada = sum(fac_tri, na.rm = TRUE)) %>%
  ungroup()

In [110]:
vab_cdmx <- vab_raw %>%
  filter(Concepto == "Ciudad de México") %>%
  pivot_longer(
    cols = `2018`:`2023 P`,
    names_to = "year",
    values_to = "vab"
  ) %>%
  mutate(year = as.integer(year)) %>%
  select(year, vab)


Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `year = as.integer(year)`.
Caused by warning:
! NAs introducidos por coerción"


In [112]:
#Calcular población ocupada anual en CDMX desde microdatos
ocupados_anual <- sdem_todos %>%
  filter(ent == 9, clase2 == 1, !is.na(fac_tri)) %>%
  group_by(year = anio) %>%
  summarise(pob_ocupada = sum(fac_tri, na.rm = TRUE)) %>%
  ungroup()


In [ ]:
#Unir VAB y población ocupada, calcular productividad e IPL
df_ipl_vab <- left_join(vab_cdmx, ocupados_anual, by = "year") %>%
  mutate(
    productividad = vab / pob_ocupada,
    base_2018 = productividad[year == 2018],
    IPL = (productividad / base_2018) * 100
  )


In [ ]:
Mostrar resultados finales
print(df_ipl_vab %>% select(year, vab, pob_ocupada,

fin